### Combination of Test 6 % 7
AI agent with memory

In [ ]:
import os
import sys
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.chat_message_histories.upstash_redis import (UpstashRedisChatMessageHistory)

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_8.ipynb')
git_add(file_path)
git_commit('Updated test 8')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

Error executing command: ['git', 'add', 'path_to_your_file']
Error message: fatal: pathspec 'path_to_your_file' did not match any files

Failed to add path_to_your_file to git.


True